In [1]:
from torch.nn import Linear, ReLU
import torch.optim as optim
# from torch_geometric.nn import Sequential, GCNConv
import torch
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
#from scipy import sparse

In [2]:
#read in the data, get normalized adjacency (NxN)
%run helpers_preproc.ipynb
mesh_dir = 'SHREC11/'

label_np = np.array(readLbl(mesh_dir+'labels.txt'))
label_mat_np = np.where(igl.all_pairs_distances(label_np,label_np,False) > 0.5,0,1)
label_mat = torch.tensor(label_mat_np,requires_grad=False)

In [3]:
adjMats = []

for i in range(600):
    fName = 'T' + str(i) + '.obj'
    adj_noscale = obj_2_adj_noscale(mesh_dir + fName)
    adj = obj_2_adj(mesh_dir + fName)
    adj_exp = np.exp(-adj) * adj_noscale
    adj = adj_exp
    adj = adj_noscale
    adj = adj / np.reshape(np.sum(adj,axis = 1),[adj.shape[0],1])
    if (adj.shape[0] <252):
        tempAdj = np.empty([252,252])
        tempAdj[:adj.shape[0],:adj.shape[0]] = adj
        adjMats.append(tempAdj)
    else:
        adjMats.append(adj)

adjMats = np.array(adjMats)

In [4]:
adjMats[0]

array([[0.        , 0.2       , 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.16666667, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.14285714,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.16666667, 0.        ,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.        , 0.14285714,
        0.        ]])

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, widths):
        super().__init__()
        self.f = 1
        self.widths = widths
        self.weights = []
        self.weights.append(torch.autograd.Variable(torch.rand(size = [self.f,widths[0]])-0.5,requires_grad = True))

        for i in range(1,len(self.widths)):
            c = self.widths[i]
            self.weights.append(torch.autograd.Variable(torch.rand((self.widths[i-1],self.widths[i]))-0.5,requires_grad = True))

    def forward(self, adjM,node_sigM):
        curr_output = torch.tanh(torch.matmul(torch.matmul(adjM,node_sigM),self.weights[0])) #nxc
        for i in range(1,len(self.widths)):
            curr_output = torch.tanh(torch.matmul(torch.matmul(adjM,curr_output),self.weights[i])) #nxc

        return curr_output

In [6]:
class NNN(torch.nn.Module):
    def __init__(self, widths):
        super().__init__()
        self.input_shape = 25200
        self.widths = widths
        self.weights = []
        self.weights.append(torch.autograd.Variable(torch.rand(size = [self.input_shape,widths[0]])-0.5,requires_grad = True))

        for i in range(1,len(self.widths)):
            c = self.widths[i]
            self.weights.append(torch.autograd.Variable(torch.rand((self.widths[i-1],self.widths[i]))-0.5,requires_grad = True))

    def forward(self, x):
        x = torch.flatten(x,1,2)
        torch.matmul(x,self.weights[0])
        curr_output = torch.tanh(torch.matmul(x,self.weights[0])) #nxc
        for i in range(1,len(self.widths)):
            curr_output = torch.tanh(torch.matmul(curr_output,self.weights[i])) #nxc

        return curr_output

In [7]:
adj = torch.tensor(adjMats, requires_grad= False).float()
node_sig = torch.tensor(np.ones([600,252,1]),requires_grad= False).float()

check = torch.sum(torch.isnan(adj))
check

tensor(0)

In [8]:
g = GCN([100])
n = NNN([100])
output1 = g.forward(adj,node_sig)
output2 = n.forward(output1)
print(output2)

tensor([[ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000],
        [ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000],
        [ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000],
        ...,
        [ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000],
        [ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000],
        [ 0.9974,  1.0000, -1.0000,  ...,  1.0000, -1.0000, -1.0000]],
       grad_fn=<TanhBackward0>)


In [24]:
def lossF(features):
    print(features)
    pdist = torch.nn.PairwiseDistance(p=2)
    pairDist = pdist(features,features)
    print(pairDist)
    return  -torch.mean(pairDist)

In [28]:
optimizer = optim.Adam(g.weights + n.weights,lr = 0.001)

for i in range(10):
    optimizer.zero_grad()
    output = n.forward(g.forward(adj,node_sig))
    loss = lossF(output)
    loss.backward()
    optimizer.step()
    print(loss)

TypeError: Adam.__init__() got multiple values for argument 'lr'

In [26]:
print(g.weights[0].grad)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])


In [ ]:
mat = np.empty([30,30])
plt.matshow(mat)
self_mean = np.mean(compSelf)
self_std = np.std(compSelf)
others_mean = np.mean(compOthers)
others_std = np.std(compOthers)

In [200]:


c1 = 10
c2 = 20
c3 = 30

weights_1 = torch.rand(size = [1,c1], requires_grad= True)
weights_2 = torch.rand([c1,c2],requires_grad= True)
weights_3 = torch.rand([c2,c3], requires_grad= False)

In [201]:
def GCN(adjM,node_sigM):
    layer_1_output = torch.matmul(torch.matmul(adjM,node_sigM),weights_1) #nxc
    #layer 2
    layer_2_output = torch.matmul(torch.matmul(adj,layer_1_output),weights_2) #nxc
    #layer 2
    layer_3_output = torch.matmul(torch.matmul(adj,layer_2_output),weights_3) #nxc
    return layer_3_output

In [202]:

optimizer = optim.Adam([weights_1,weights_2,weights_3],lr = 0.01)

for i in range(100):
    optimizer.zero_grad()
    output = GCN(adj,node_sig)
    #output = torch.mean(weights_1)
    loss = torch.mean(output)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(439.9774, grad_fn=<MeanBackward0>)
tensor(421.3802, grad_fn=<MeanBackward0>)
tensor(403.1801, grad_fn=<MeanBackward0>)
tensor(385.3835, grad_fn=<MeanBackward0>)
tensor(367.9957, grad_fn=<MeanBackward0>)
tensor(351.0208, grad_fn=<MeanBackward0>)
tensor(334.4612, grad_fn=<MeanBackward0>)
tensor(318.3172, grad_fn=<MeanBackward0>)
tensor(302.5864, grad_fn=<MeanBackward0>)
tensor(287.2630, grad_fn=<MeanBackward0>)
tensor(272.3385, grad_fn=<MeanBackward0>)
tensor(257.8025, grad_fn=<MeanBackward0>)
tensor(243.6440, grad_fn=<MeanBackward0>)
tensor(229.8531, grad_fn=<MeanBackward0>)
tensor(216.4211, grad_fn=<MeanBackward0>)
tensor(203.3407, grad_fn=<MeanBackward0>)
tensor(190.6051, grad_fn=<MeanBackward0>)
tensor(178.2077, grad_fn=<MeanBackward0>)
tensor(166.1418, grad_fn=<MeanBackward0>)
tensor(154.4001, grad_fn=<MeanBackward0>)
tensor(142.9748, grad_fn=<MeanBackward0>)
tensor(131.8571, grad_fn=<MeanBackward0>)
tensor(121.0378, grad_fn=<MeanBackward0>)
tensor(110.5065, grad_fn=<MeanBack

In [249]:
a = torch.tensor([450,500,300])
torch.tanh(a)

tensor([1., 1., 1.])

In [203]:
# adj_tensor = torch.tensor(adjMats)
# edge_indx = adj_tensor.nonzero().t().contiguous()

In [207]:
# # print(edge_indx)
# # print(edge_indx.shape)
# print(torch.max(edge_indx))
# print(edge_indx.size())

In [201]:
# data = torch_geometric.data.Data(x = torch.zeros(252,1), edge_index=edge_indx)

In [202]:
# #hyper parameters
# in_channels = 252
# out_channels = 10

In [203]:
# #set up gnn
# model = Sequential('x, edge_index', [
#     (GCNConv(in_channels, 64), 'x, edge_index -> x'),
#     ReLU(inplace=True),
#     (GCNConv(64, 64), 'x, edge_index -> x'),
#     ReLU(inplace=True),
#     Linear(64, out_channels),
# ])

In [208]:
# data = torch.tensor(adjMats).float()
#
# model(data,edge_indx)


In [ ]:
#run gnn, get (NxC) matrix


In [ ]:
#compute contrastive loss


In [ ]:
#update weights (FxC)
